# Remote Query Examples

This notebook demonstrates remote querying capabilities using `SPARQLWrapper`

SPARQLWrapper is a Python wrapper around a SPARQL service that gives the capability to
remotely execute queries on SPARQL endpoints. It simply invokes a SPARQLWrapper class
that requires an endpoint which you can pass a query to. The following are a few
examples that demonstate the capabilities:

In [ ]:
import ipywidgets as W
import pandas as pd
import traitlets as T
from rdflib import BNode, Graph, Literal, URIRef, namespace
from SPARQLWrapper import JSON, XML, SPARQLWrapper

from ipyradiant import (
    CytoscapeVisualizer,
    FileManager,
    PathLoader,
    QueryWidget,
    service_patch_rdflib,
)

In [ ]:
try:
    import requests

    resp = requests.get("http://dbpedia.org/sparql/")
    assert resp.ok
    DBPEDIA_AVAILABLE = True
except AssertionError:
    print("DBPedia not available (404).")
    DBPEDIA_AVAILABLE = False

### Query a single remote endpoint using `SPARQLWrapper`

This example creates a sparql class from the graph location: http://dbpedia.org/sparql.
In the setQuery method, you can pass a string that describes the query you want. Here,
we perform a SELECT query that returns the labels for the resource Asturias from
dbpedia:

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql/")
sparql.setQuery(
    """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Asturias> rdfs:label ?label }
"""
)

if DBPEDIA_AVAILABLE:
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    results_df = pd.json_normalize(results["results"]["bindings"])
    results_df[["label.value"]]

> Note: `SPARQLWrapper.setQuery` supports various query methods such as SELECT, ASK,
> CONSTRUCT, and DESCRIBE

### SERVICE patch for rdflib

Before we look at remote query examples with `SERVICE` calls, let's discuss an issue
with rdflib:

Currently, rdflib contains a bug where the SERVICE clause is not supported properly.
ipyradiant detects when SERVICE is used for federated queries and converts the keyword
into lower case for rdflib support. A warning is issued when SERVICE is detected. This
patch is turned off for release>5.0.0

Here is a working example of the query string conversion to a scheme that is supported
by rdflib:

In [ ]:
query_str = """
    SELECT DISTINCT ?s ?p ?o
    WHERE
      { 
        SERVICE <http://dbpedia.org/sparql> 
          {
            SELECT ?s ?p ?o
            WHERE {?s ?p ?o}               
          }
      }
"""

if DBPEDIA_AVAILABLE:
    query_str = service_patch_rdflib(query_str)
    print(query_str)

### Query Widget Example

Here we show a working example of Remote Querying using the Query Widget available under
ipyradiant. First we define a WidgetExample class and fill in the query form with query
that contains a SERVICE call:

In [ ]:
class WidgetExample(W.Tab):
    query = T.Instance(QueryWidget)
    log = W.Output()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.children = [self.query]

    @T.default("query")
    def make_default_query_widget(self):
        return QueryWidget()

The following code snippet creates a Widget UI where you can see the query being passed
in. Take a look at the UI panel on the right where you can see the query using the
SPARQL endpoint from linkeddata. If you click on `Run Query`, you can immediate see the
query string being modified to the correct form:

In [ ]:
widget = WidgetExample()
widget.query.query = """\
SELECT DISTINCT *
WHERE {
 SERVICE <http://dbpedia.org/sparql>
    {
        SELECT ?s ?p ?o
        WHERE {?s ?p ?o}
        LIMIT 10
    }
}
"""

In [ ]:
# "click" the button
if DBPEDIA_AVAILABLE:
    widget.query.run_button.click()

### Nested Query Example

A known issue with rdflib is that it does not support nested service calls in its query
body. An example of this can be seen in the
<a href="examples/FederatedQuery_Example.ipynb">Federated Query Examples</a> notebook.
We can utilize SPARQLWrapper to perform the same task. The following is a working
example of utlizing two separate SERVICE calls and combining the results:

In [ ]:
agent = "Chrome/86.0.4240.198"  # default SPARQLWrapper agent does not work for wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=agent)
sparql.setQuery(
    """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?p ?item
WHERE {
    
    SELECT ?p ?item
    WHERE 
    {
      BIND(wikibase:label as ?p)
      wd:Q28792126 wdt:P31 wd:Q146 .
      
      service <https://query.wikidata.org/sparql>
      { 
        SELECT ?item
        WHERE {
            ?item wdt:P31 wd:Q146 .
        }
        LIMIT 5
      }
    } 
    
    
}
"""
)
sparql.setReturnFormat(JSON)
if DBPEDIA_AVAILABLE:
    results = sparql.query().convert()
    results["results"]["bindings"]

    results_df = pd.json_normalize(results["results"]["bindings"])
    results_df

### Example using Wikidata and in-built service call

In this example, we use SPARQLWrapper to query Wikidata via the Wikidata Query Service
and organize the output response using pandas:

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=agent)
sparql.setQuery(
    """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?item ?itemLabel
WHERE
{
    ?item wdt:P31 wd:Q146 .
    service wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""
)
sparql.setReturnFormat(JSON)
if DBPEDIA_AVAILABLE:
    results = sparql.query().convert()
    results_df = pd.json_normalize(results["results"]["bindings"])
    results_df[["item.value"]].head()

For more examples and usage of SPARQLWrapper, please visit the Github repo found here:
https://github.com/RDFLib/sparqlwrapper